# Define processing functions
This notebook contains functions used to process raw model output. 
- Wanninkhof92
- calculate_flux
- 

## wanninkhof92
Python implementation of Wanninkhof (1992) parameterization with Sweeney et al. (2007) scale factor. Atmospheric CO2 is calculated in moist air with the saturation vapor pressure calculated following Dickson et al. (2007); although Weiss (1974) has a different functional form, the two parameterizations agree quite well.

In [1]:
import xarray.ufuncs as xu

In [ ]:
def wanninkhof92(T=None, 
                 S=None, 
                 P=None,
                 u_mean=None, 
                 u_std=None, 
                 xCO2=None,
                 pCO2_sw = None,
                 iceFrac = None,
                 scale_factor=0.27):
    '''
    Calculate air-sea CO2 flux following Wanninkhof 1992
    
    Inputs
    ============
    T = Temperature [degC]
    S = Salinity  [parts per thousand]
    P = Atmospheric pressure at 10m [atm]
    u_mean = mean wind speed [m/s]
    u_std = standard deviation of wind speed m/s averaged monthly [m/s]
    xcO2 = atmoapheric mixing ratio of CO2 [ppm]
    pCO2_sw = CO2 partial pressure of seawter [uatm]
    scale_factor = gas transfer scale factor (default=0.27)
    
    Output
    ============
    F_co2 = air-sea co2 flux [molC/m2/yr]
    
    References
    ============
    Weiss (1974) [for solubility of CO2]
        Carbon dioxide in water and seawater: the solubility of a non-ideal gas
    Weiss and Price (1980) [for saturation vapor pressure]
        Nitrous oxide solubility in water and seawater     
    Dickson et al (2007) [for saturation vapor pressure]
        Guide to best practices for Ocean CO2 measurements, Chapter 5 section 3.2
    Wanninkhof (1992) [for gas transfer and Schmidt number]
        Relationship between wind speed and gas exchange over the ocean   
    Sweeney et al. (2007) [for gas transfer scale factor]
        Constraining global air‐sea gas exchange for CO2 with recent bomb 14C measurements
    Sarmiento and Gruber (2007) [for overview and discussion of CO2 flux]
        Ocean biogeochmiecal dynsmics, Ch3 see panel 3.2.1
    
    Notes
    ============
    this code is optimized to work with Xarray
    
    ## Notes on partial pressure moist air
    P_h20/P = saturation vapor pressure (Weiss and Price 1980)
    Water vapor is generally assumed to be at saturation in the vicinity of the air-sea interfae.
        p_moist = X_dry (P - P_h20)
                = x_dry * P (1-P_h20/P)
                = P_dry (1-P_h20/P)

    ## Notes on U2
    Need to add wind speed variance if using Wanninkhof (1992)
    If we don't, then we are underestimating the gas-transfer velocity
    U2 = (U_mean)^2 + (U_prime)^2, (U_prime)^2 is the standard deviation
    See Sarmiento and Gruber (2007) for a discussion of this
    
    ## Test values
    # T = 298.15
    # S = 35
    
    '''
    
    ### ================================================
    ### 0. Conversions
    ### ================================================
    ### Convert from Celsius to kelvin
    T_kelvin = T + 273.15
    
    
    ### ================================================
    ### 1. partial pressure of CO2 in moist air
    ###    Units : uatm
    ### ================================================    
    ### Weiss and Price (1980) formula
    # P_sat_vapor = xu.exp(24.4543 - 67.4509*(100/T_kelvin) - 4.8489*xu.log(T_kelvin/100) - 0.000544*S)
    
    
    ### Following Dickson et al (2007) Chapter 5 section 3.2
    a1 = -7.85951783
    a2 = 1.84408259
    a3 = -11.7866497
    a4 = 22.6807411
    a5 = -15.9618719
    a6 = 1.80122502

    Tc = 647.096
    pc = (22.064 * 10**6) / 101325
    g = (1-(T_kelvin/Tc))

    ### pure water saturation vapor pressure, Wagner and Pruß, 2002
    p_h20 = pc * xu.exp((Tc/T_kelvin) * (a1*g + 
                          a2*g**(1.5) + 
                          a3*g**(3) + 
                          a4*g**(3.5) + 
                          a5*g**(4) + 
                          a6*g**(7.5)))


    ### total molality 
    MT = (31.998*S) / (1000 - 1.005*S)

    ### osmotic coefficient at 25C by Millero (1974)
    b1 = 0.90799 
    b2 = - 0.08992
    b3 = 0.18458
    b4 = -0.07395
    b5 = -0.00221
    phi = b1 + b2*(0.5*MT) + b3*(0.5*MT)**2 + b4*(0.5*MT)**3 + b5*(0.5*MT)**4

    ### vapor pressure above sea-water, Dickson et al. (2007), Chapter 5, section 3.2
    P_sat_vapor = p_h20 * xu.exp(-0.018*phi*MT)

    ### Partial pressure of CO2 in moist air, Dickson et al. (2007), SOP 4 section 8.3
    #P_moist = (P * xCO2) * (1 - P_sat_vapor)
    P_moist = xCO2 * (P - P_sat_vapor)
    
    ### ================================================
    ### 2. Solubility of CO2
    ###    Notes : T in degC, S in PSU, 
    ###            1.0E-6 converts to correct to muatm
    ###    Units : to mol.kg^{-1}uatm^{-1}
    ###    Reference : Weiss (1974)
    ### ================================================  
    S_co2 = xu.exp( 9345.17 / T_kelvin - 60.2409 + \
                     23.3585 * xu.log( T_kelvin / 100 ) + \
                     S * ( 0.023517 - 0.00023656 * T_kelvin + \
                          0.0047036 * ( T_kelvin / 100 )**2 )) *1.0E-6
    
    
    ### ================================================
    ### 3. Gas transfer velocity
    ###    Units : cm/hr
    ###    Reference : Wanninkhof (1992)
    ###                Sweeney et al. (2007)
    ###                per.comm with P. Landschutzer Feb. 19 2019
    ### ================================================  
    # Dimensionless Schmidt number (Sc)
    # References Wanninkhof 1992
    A = 2073.1 
    B = 125.62 
    C = 3.6276 
    D = 0.043219
    
    # Schmidt number 
    # units : dimensionless
    Sc = A - B*T + C*T**2 - D*T**3
    
    # Gas transfer velocity
    # References :  Wanninkhof 1992, 
    # Sweeney et al. 2007 scale factor
    k_w = scale_factor * (Sc/660)**(-0.5) * (u_mean**2 + u_std)

    
    ### ================================================
    ### 4. air-sea CO2 exchange
    ###    Units : mol/m2/yr
    ###    Reference : Wanninkhof (1992)
    ### ================================================  
    # Convert from cm*mol/hr/kg to mol/m2/yr
    conversion = (1000/100)*365*24
    
    # Air-sea CO2 flux 
    F_co2 = k_w * S_co2 * (1 - iceFrac) * (pCO2_sw - P_moist) * conversion
    
    return F_co2

## calculate_flux
This function calculates the air-sea CO2 exchange following Wanninkhof (1992) using the modeled pCO2 and the reconstructed pCO2. This stores the output as a NetCDF file in the {data_dir} location

In [ ]:
def calculate_flux_float(model=None, member=None, fl_u10_std=None, dir_out=None):
    '''
    calculate_flux(model=None, member=None)
    calculate CO2 flux following Wanninkhof (1992) for each member.
    
    Inputs
    ============
    model  = name of model (CESM, CanESM2, GFDL, MPI)
    member = member number
    fl_u10_std = file path to u10-STD netcdf file
    dir_out = directory to store output (do not put "/" at end)
    
    Output
    ============
    None. This stores the calculated flux as a NetCDF file in the {data_dir} location.
    
    Notes
    ============
    This uses the 
    
    
    _model2() function contained in _define_model_class.ipynb
    This uses the scaled version of Sweeney et al. (2007) used in 
    Landschutzer et al. (2016). Scale factor 1.014 is from person comm. with Peter
    '''
    ###======================================
    ### Define directories
    ###====================================== 
    #dir_raw = '/local/data/artemis/workspace/gloege/SOCAT-LE/data/raw'
    #dir_clean = '/local/data/artemis/workspace/gloege/SOCAT-LE/data/clean'
    
    ### ================================================
    ### Get file path
    ### ================================================
    if model.upper()=='CESM':
        mem = f'{member:0>3}'
    if model.upper()=='GFDL':
        mem = f'{member:0>2}'
    if model.upper()=='MPI':
        mem = f'{member:0>3}'
    if model.upper()=='CANESM2':
        mem = f'{member}'

    ### Load auxillary data
    ds_SST     = read_model2(model=model, member=mem, variable='SST')
    ds_SSS     = read_model2(model=model, member=mem, variable='SSS')
    ds_iceFrac = read_model2(model=model, member=mem, variable='iceFrac')
    ds_pATM    = read_model2(model=model, member=mem, variable='pATM')
    ds_XCO2    = read_model2(model=model, member=mem, variable='XCO2')
    ds_U10     = read_model2(model=model, member=mem, variable='U10')
    ds_pCO2    = read_model2(model=model, member=mem, variable='pCO2')
    
    ds_pCO2_SOMFFN = read_somffn_float(model=model, member=mem)
    
    ### Wind speed variance 
    #ds_U_std = xr.open_dataset(f'{dir_clean}/ERA_interim/ERAinterim_1x1_u10-std_1982-2016.nc', 
    #                           decode_times=False)
    ds_U_std = xr.open_dataset(f'{fl_u10_std}', decode_times=False)
    ###======================================
    ### Put data into xarray dataset
    ###======================================
    ds = xr.Dataset(
        {
        'SST':(['time','lat','lon'], ds_SST['SST'] ),
        'SSS':(['time','lat','lon'], ds_SSS['SSS']),
        'iceFrac':(['time','lat','lon'], ds_iceFrac['iceFrac'] ),
        'U10':(['time','lat','lon'], ds_U10['U10']),
        'U10_std':(['time','lat','lon'], ds_U_std['u10_var']),
        'Patm':(['time','lat','lon'], ds_pATM['pATM'] ),
        'xCO2':(['time'], ds_XCO2['XCO2']),
        'pCO2_member':(['time','lat','lon'], ds_pCO2['pCO2']),
        'pCO2_somffn':(['time','lat','lon'], ds_pCO2_SOMFFN['pco2']),
        },

        coords={
        'lat': (['lat'], ds_pCO2_SOMFFN['lat']),
        'lon': (['lon'], ds_pCO2_SOMFFN['lon']),
        'time': (['time'], ds_pCO2_SOMFFN['time'])
        })

    F_somffn = wanninkhof92(T=ds['SST'], 
                 S=ds['SSS'], 
                 P=ds['Patm'],
                 u_mean=ds['U10'], 
                 u_std=ds['U10_std'], 
                 xCO2=ds['xCO2'],
                 pCO2_sw = ds['pCO2_somffn'],
                 iceFrac = ds['iceFrac'],
                 scale_factor=0.27*1.014)
        
    F_member = wanninkhof92(T=ds['SST'], 
                 S=ds['SSS'], 
                 P=ds['Patm'],
                 u_mean=ds['U10'], 
                 u_std=ds['U10_std'], 
                 xCO2=ds['xCO2'],
                 pCO2_sw = ds['pCO2_member'],
                 iceFrac = ds['iceFrac'],
                 scale_factor=0.27*1.014)

    # Save output in file
    ds_out = xr.Dataset(
        {
        'F_member':(['time','lat','lon'], F_member), 
        'F_somffn':(['time','lat','lon'], F_somffn),       
        },

        coords={
        'lat': (['lat'], ds['lat']),
        'lon': (['lon'], ds['lon']),
        'time': (['time'], ds['time'])
        })
        
    # Save to netcdf
    ds_out.to_netcdf(f'{dir_out}/CO2_flux_{model}{mem}_SOMFFN.nc')

In [ ]:
def calculate_flux(model=None, member=None, fl_u10_std=None, dir_out=None, ):
    '''
    calculate_flux(model=None, member=None)
    calculate CO2 flux following Wanninkhof (1992) for each member.
    
    Inputs
    ============
    model  = name of model (CESM, CanESM2, GFDL, MPI)
    member = member number
    fl_u10_std = file path to u10-STD netcdf file
    dir_out = directory to store output (do not put "/" at end)
    
    Output
    ============
    None. This stores the calculated flux as a NetCDF file in the {data_dir} location.
    
    Notes
    ============
    This uses the read_model2() function contained in _define_model_class.ipynb
    This uses the scaled version of Sweeney et al. (2007) used in 
    Landschutzer et al. (2016). Scale factor 1.014 is from person comm. with Peter
    '''
    ###======================================
    ### Define directories
    ###====================================== 
    #dir_raw = '/local/data/artemis/workspace/gloege/SOCAT-LE/data/raw'
    #dir_clean = '/local/data/artemis/workspace/gloege/SOCAT-LE/data/clean'
    
    ### ================================================
    ### Get file path
    ### ================================================
    if model.upper()=='CESM':
        mem = f'{member:0>3}'
    if model.upper()=='GFDL':
        mem = f'{member:0>2}'
    if model.upper()=='MPI':
        mem = f'{member:0>3}'
    if model.upper()=='CANESM2':
        mem = f'{member}'

    ### Load auxillary data
    ds_SST     = read_model2(model=model, member=mem, variable='SST')
    ds_SSS     = read_model2(model=model, member=mem, variable='SSS')
    ds_iceFrac = read_model2(model=model, member=mem, variable='iceFrac')
    ds_pATM    = read_model2(model=model, member=mem, variable='pATM')
    ds_XCO2    = read_model2(model=model, member=mem, variable='XCO2')
    ds_U10     = read_model2(model=model, member=mem, variable='U10')
    ds_pCO2    = read_model2(model=model, member=mem, variable='pCO2')
    ds_pCO2_SOMFFN = read_somffn(model=model, member=mem)
    
    ### Wind speed variance 
    #ds_U_std = xr.open_dataset(f'{dir_clean}/ERA_interim/ERAinterim_1x1_u10-std_1982-2016.nc', 
    #                           decode_times=False)
    ds_U_std = xr.open_dataset(f'{fl_u10_std}', decode_times=False)
    ###======================================
    ### Put data into xarray dataset
    ###======================================
    ds = xr.Dataset(
        {
        'SST':(['time','lat','lon'], ds_SST['SST'] ),
        'SSS':(['time','lat','lon'], ds_SSS['SSS']),
        'iceFrac':(['time','lat','lon'], ds_iceFrac['iceFrac'] ),
        'U10':(['time','lat','lon'], ds_U10['U10']),
        'U10_std':(['time','lat','lon'], ds_U_std['u10_var']),
        'Patm':(['time','lat','lon'], ds_pATM['pATM'] ),
        'xCO2':(['time'], ds_XCO2['XCO2']),
        'pCO2_member':(['time','lat','lon'], ds_pCO2['pCO2']),
        'pCO2_somffn':(['time','lat','lon'], ds_pCO2_SOMFFN['pco2']),
        },

        coords={
        'lat': (['lat'], ds_pCO2_SOMFFN['lat']),
        'lon': (['lon'], ds_pCO2_SOMFFN['lon']),
        'time': (['time'], ds_pCO2_SOMFFN['time'])
        })

    F_somffn = wanninkhof92(T=ds['SST'], 
                 S=ds['SSS'], 
                 P=ds['Patm'],
                 u_mean=ds['U10'], 
                 u_std=ds['U10_std'], 
                 xCO2=ds['xCO2'],
                 pCO2_sw = ds['pCO2_somffn'],
                 iceFrac = ds['iceFrac'],
                 scale_factor=0.27*1.014)
        
    F_member = wanninkhof92(T=ds['SST'], 
                 S=ds['SSS'], 
                 P=ds['Patm'],
                 u_mean=ds['U10'], 
                 u_std=ds['U10_std'], 
                 xCO2=ds['xCO2'],
                 pCO2_sw = ds['pCO2_member'],
                 iceFrac = ds['iceFrac'],
                 scale_factor=0.27*1.014)

    # Save output in file
    ds_out = xr.Dataset(
        {
        'F_member':(['time','lat','lon'], F_member), 
        'F_somffn':(['time','lat','lon'], F_somffn),       
        },

        coords={
        'lat': (['lat'], ds['lat']),
        'lon': (['lon'], ds['lon']),
        'time': (['time'], ds['time'])
        })
        
    # Save to netcdf
    ds_out.to_netcdf(f'{dir_out}/CO2_flux_{model}{mem}_SOMFFN.nc')

# calculate_flux_statistics
This function calculates statistics on each member and stores each one separately.

In [ ]:
def calculate_flux_statistics(model=None, member=None, dir_in=None, dir_out=None):
    """
    calculate_flux_statistics(model=None, member=None)
    model   = [MPI, CESM, or GFDL]
    member  = "member you want to process"
    dir_in  = directory for CO2 inputs (do not put / at end)
    dir_out = directory for output (do not put / at end) 
    
    Notes
    =========
    
    L. Gloege 2018
    """
    
    ### ================================================
    ### Get file path
    ### ================================================
    if model.upper()=='CESM':
        mem = f'{member:0>3}'
    if model.upper()=='GFDL':
        mem = f'{member:0>2}'
    if model.upper()=='MPI':
        mem = f'{member:0>3}'
    if model.upper()=='CANESM2':
        mem = f'{member}'
        
    ###======================================
    ### Load model
    ###====================================== 
    ds_som = xr.open_dataset(f'{dir_in}/CO2_flux_decomp_{model}{mem}_SOMFFN.nc')
    ds_mod = xr.open_dataset(f'{dir_in}/CO2_flux_decomp_{model}{mem}_MODEL.nc')
        
    ###======================================
    ### Return dataset
    ###======================================
    ds_out = xr.Dataset(
        {
        ## Bias
        'bias': (['lat', 'lon'], sk.avg_error(ds_mod['F_member'], 
                                              ds_som['F_somffn']),
                 {'units':'mol/m2/yr'}), 
        'bias_detrend':(['lat','lon'], sk.avg_error(ds_mod['F_member_detrend'], 
                                                    ds_som['F_somffn_detrend']),
                       {'units':'mol/m2/yr'}),
        'bias_dec': (['lat', 'lon'], sk.avg_error(ds_mod['F_member_dec'], 
                                                    ds_som['F_somffn_dec']),
                       {'units':'mol/m2/yr'}),
        'bias_seasonal': (['lat', 'lon'], sk.avg_error(ds_mod['F_member_seasonal'], 
                                                      ds_som['F_somffn_seasonal']),
                       {'units':'mol/m2/yr'}),
        'bias_residual': (['lat', 'lon'], sk.avg_error(ds_mod['F_member_residual'], 
                                                      ds_som['F_somffn_residual']),
                       {'units':'mol/m2/yr'}),
        'bias_residual_low': (['lat', 'lon'], sk.avg_error(ds_mod['F_member_residual_low'], 
                                                      ds_som['F_somffn_residual_low']),
                       {'units':'mol/m2/yr'}),

        ## average absolute error
        'aae': (['lat', 'lon'], sk.avg_abs_error(ds_mod['F_member'], 
                                              ds_som['F_somffn']),
                 {'units':'mol/m2/yr'}), 
        'aae_detrend':(['lat','lon'], sk.avg_abs_error(ds_mod['F_member_detrend'], 
                                                    ds_som['F_somffn_detrend']),
                       {'units':'mol/m2/yr'}),
        'aae_dec': (['lat', 'lon'], sk.avg_abs_error(ds_mod['F_member_dec'], 
                                                    ds_som['F_somffn_dec']),
                       {'units':'mol/m2/yr'}),
        'aae_seasonal': (['lat', 'lon'], sk.avg_abs_error(ds_mod['F_member_seasonal'], 
                                                      ds_som['F_somffn_seasonal']),
                       {'units':'mol/m2/yr'}),
        'aae_residual': (['lat', 'lon'], sk.avg_abs_error(ds_mod['F_member_residual'], 
                                                      ds_som['F_somffn_residual']),
                       {'units':'mol/m2/yr'}),
        'aae_residual_low': (['lat', 'lon'], sk.avg_abs_error(ds_mod['F_member_residual_low'], 
                                                      ds_som['F_somffn_residual_low']),
                       {'units':'mol/m2/yr'}),
            
       
        ## Amplitude ratio
        'amp-ratio': (['lat', 'lon'], sk.amp_ratio(ds_mod['F_member'], 
                                              ds_som['F_somffn']),
              {'units':'dimensionless',
               'notes':'[(amp_rec/amp_mod) - 1]'}),
        'amp-ratio_detrend':(['lat','lon'], sk.amp_ratio(ds_mod['F_member_detrend'], 
                                                    ds_som['F_somffn_detrend']),
              {'units':'dimensionless',
               'notes':'[(amp_rec/amp_mod) - 1]'}),
        'amp-ratio_dec': (['lat', 'lon'], sk.amp_ratio(ds_mod['F_member_dec'], 
                                                    ds_som['F_somffn_dec']),
              {'units':'dimensionless',
               'notes':'[(amp_rec/amp_mod) - 1]'}),
        'amp-ratio_seasonal': (['lat', 'lon'], sk.amp_ratio(ds_mod['F_member_seasonal'], 
                                                      ds_som['F_somffn_seasonal']),
              {'units':'dimensionless',
               'notes':'[(amp_rec/amp_mod) - 1]'}),
        'amp-ratio_residual': (['lat', 'lon'], sk.amp_ratio(ds_mod['F_member_residual'], 
                                                      ds_som['F_somffn_residual']),
              {'units':'dimensionless',
               'notes':'[(amp_rec/amp_mod) - 1]'}),
        'amp-ratio_residual_low': (['lat', 'lon'], sk.amp_ratio(ds_mod['F_member_residual_low'], 
                                                      ds_som['F_somffn_residual_low']),
              {'units':'dimensionless',
               'notes':'[(amp_rec/amp_mod) - 1]'}),
            
        
        ## correlation
        'corr': (['lat', 'lon'], sk.correlation(ds_mod['F_member'], 
                                              ds_som['F_somffn']),
            {'units':'dimensionless'}),
        'corr_detrend':(['lat','lon'], sk.correlation(ds_mod['F_member_detrend'], 
                                                    ds_som['F_somffn_detrend']),
            {'units':'dimensionless'}),
        'corr_dec': (['lat', 'lon'], sk.correlation(ds_mod['F_member_dec'], 
                                                    ds_som['F_somffn_dec']),
            {'units':'dimensionless'}),
        'corr_seasonal': (['lat', 'lon'], sk.correlation(ds_mod['F_member_seasonal'], 
                                                      ds_som['F_somffn_seasonal']),
            {'units':'dimensionless'}),
        'corr_residual': (['lat', 'lon'], sk.correlation(ds_mod['F_member_residual'], 
                                                      ds_som['F_somffn_residual']),
            {'units':'dimensionless'}),
        'corr_residual_low': (['lat', 'lon'], sk.correlation(ds_mod['F_member_residual_low'], 
                                                      ds_som['F_somffn_residual_low']),
            {'units':'dimensionless'}),
            
            
        ## Normalized standard deviation
        'std-star': (['lat', 'lon'], sk.std_star(ds_mod['F_member'], 
                                              ds_som['F_somffn']),
                      {'units':'dimensionless',
                       'notes':'[(STD_rec/STD_mod) - 1]'}),
        'std-star_detrend':(['lat','lon'], sk.std_star(ds_mod['F_member_detrend'], 
                                                    ds_som['F_somffn_detrend']),
                      {'units':'dimensionless',
                       'notes':'[(STD_rec/STD_mod) - 1]'}),
        'std-star_dec': (['lat', 'lon'], sk.std_star(ds_mod['F_member_dec'], 
                                                    ds_som['F_somffn_dec']),
                      {'units':'dimensionless',
                       'notes':'[(STD_rec/STD_mod) - 1]'}),
        'std-star_seasonal': (['lat', 'lon'], sk.std_star(ds_mod['F_member_seasonal'], 
                                                      ds_som['F_somffn_seasonal']),
                      {'units':'dimensionless',
                       'notes':'[(STD_rec/STD_mod) - 1]'}),
        'std-star_residual': (['lat', 'lon'], sk.std_star(ds_mod['F_member_residual'], 
                                                      ds_som['F_somffn_residual']),
                      {'units':'dimensionless',
                       'notes':'[(STD_rec/STD_mod) - 1]'}),
        'std-star_residual_low': (['lat', 'lon'], sk.std_star(ds_mod['F_member_residual_low'], 
                                                      ds_som['F_somffn_residual_low']),
                      {'units':'dimensionless',
                       'notes':'[(STD_rec/STD_mod) - 1]'})
        },

        coords={
        'lat': (['lat'], ds_mod['lat']),
        'lon': (['lon'], ds_mod['lon']),
        })
        
    # save file
    ds_out.to_netcdf(f'{dir_out}/stats_FCO2_{model}{mem}_SOMFFN.nc') 

# decompose_pco2 -- STL
This function decomposes the pCO2 into different time-scales using the STL method.

1. detrended  the data - removes a linear trend
2. Calculate repeating seasonal cycle and remove it
3. Calculate trend using LOWESS - calculates with 5 years of data
4. With the remainder, calculate low frequency variability

In [ ]:
def decompose_pco2(model=None, member=None, model_or_recon=None, dir_out=None):
    '''
    decompose_pco2
    this function decomposes pCO2 into various time scales
    
    Inputs
    ============
    model  = name of model (CESM, CanESM2, GFDL, MPI)
    member = member number
    model_or_recon = either SOMFFN or MODEL
    
    Output
    ============
    None. This stores the calculated flux as a NetCDF file in the {data_dir} location.
    
    Notes
    ============
    This assumes ERA interm is here:
    /local/data/artemis/workspace/gloege/SOCAT-LE/data/clean/ERA_interim/
    You will need to change this if this is not correct
    
    and data is output to this directory:
    /local/data/artemis/workspace/gloege/SOCAT-LE/data/clean/CO2_flux/
    
    '''
    ###======================================
    ### Define directories
    ###====================================== 
    #dir_clean = '/local/data/artemis/workspace/gloege/SOCAT-LE/data/clean'
    
    ###======================================
    ### Load data
    ###======================================
    print(model_or_recon)
    if model_or_recon.upper()=='SOMFFN':
        ds_model = read_peter(model=model, member=member)
        
    if model_or_recon.upper()=='MODEL':
        ds_model = read_model(model=model, member=member)
        
    ###======================================
    ### Add padding to member number
    ###======================================
    if model.upper()=='CESM':
        mem = f'{member:0>3}'
        
    if model.upper()=='GFDL':
        mem = f'{member:0>2}'

    if model.upper()=='MPI':
        mem = f'{member:0>3}'

    if model.upper()=='CANESM2':
        mem = f'{member}'
   
    ###======================================
    ### Put data into xarray dataset
    ###======================================
    ds = xr.Dataset(
        {
        'pco2_model':(['time','lat','lon'], ds_model['pco2']),
        },

        coords={
        'lat': (['lat'], ds_model['lat']),
        'lon': (['lon'], ds_model['lon']),
        'time': (['time'], ds_model['time'])
        })
    
    ### Make sure correct order
    ds = ds.transpose('time', 'lat', 'lon')
        
    ###======================================
    ### STL Decomposition
    ###======================================
    ### 0. Load raw data
    data = ds[f'pco2_model'].copy()
    
    ### 1. Detrend it 
    data_detrend = stl.detrend(data, dim='time')
    
    ### 2. Remove seasonal cycle
    # 2.1 -- seasonal cycle
    data_seasonal = stl.seasonal_cycle(data_detrend, dim='time', period=12)
    
    # 2.2 -- de-season the data
    data_deseason = data_detrend - data_seasonal
    
    ### 3. calculate the LOWESS
    data_lowess = stl.lowess(data_deseason, dim='time', lo_pts=12*10, lo_delta=0.01)

    ### 4. Residual term -- not explained by trend or seasonal cycle 
    data_residual = data_deseason - data_lowess
    
    ### Low frequency of residual
    data_residual_low = stl.lowess(data_residual, dim='time', lo_pts=12, lo_delta=0.01)

    
    ###======================================
    ### Return dataset
    ###======================================
    ds_out = xr.Dataset(
        {    
        'pco2': (['time', 'lat', 'lon'], data ),
        'pco2_detrend': (['time', 'lat', 'lon'], data_detrend ),
        'pco2_dec': (['time','lat', 'lon'], data_lowess ),
        'pco2_seasonal': (['time','lat', 'lon'], data_seasonal ),
        'pco2_residual': (['time','lat', 'lon'], data_residual ),
        'pco2_residual_low': (['time','lat', 'lon'], data_residual_low ),
        },

        coords={
        'time': (['time'], ds_model['time']),
        'lat': (['lat'], ds_model['lat']),
        'lon': (['lon'], ds_model['lon']),
        })
        
    ### save file
    ds_out.to_netcdf(f'{dir_out}/pco2_decomp_{model}{mem}_{model_or_recon}.nc') 

# decompose_flux -- STL
This uses the old routine with moving averages .
This is not ideal because you continously 

In [ ]:
import decompose as stl
def decompose_flux(model=None, member=None, model_or_recon=None, dir_in=None, dir_out=None):
    '''
    decompose_flux
    this function decomposes flux into various time scales
    
    Inputs
    ============
    model  = name of model (CESM, CanESM2, GFDL, MPI)
    member = member number
    model_or_recon = either SOMFFN or MODEL
    dir_in  = directory for CO2 inputs (do not put / at end)
    dir_out = directory for output (do not put / at end) 
    
    Output
    ============
    None. This stores the calculated flux as a NetCDF file in the {data_dir} location.
    
    Notes
    ============
    This assumes ERA interm is here:
    /local/data/artemis/workspace/gloege/SOCAT-LE/data/clean/ERA_interim/
    You will need to change this if this is not correct
    
    and data is output to this directory:
    /local/data/artemis/workspace/gloege/SOCAT-LE/data/clean/CO2_flux/
    
    '''
    
    ###======================================
    ### This is so naming convention same as decompose_pco2
    ###====================================== 
    if model_or_recon.upper() == 'MODEL':
        member_or_somffn = 'member'
    elif model_or_recon.upper() == 'SOMFFN':
        member_or_somffn = 'somffn'
    else:
        print('model_or_recon needs to be either MODEL or SOMFFN')
        
    ###======================================
    ### Load data
    ###======================================
    if model.upper()=='CESM':
        mem = f'{member:0>3}'
        fl = f'{dir_in}/CO2_flux_CESM{mem}_SOMFFN.nc'
        
    if model.upper()=='GFDL':
        mem = f'{member:0>2}'
        fl = f'{dir_in}/CO2_flux_GFDL{mem}_SOMFFN.nc'
        
    if model.upper()=='MPI':
        mem = f'{member:0>3}'
        fl = f'{dir_in}/CO2_flux_MPI{mem}_SOMFFN.nc'

    if model.upper()=='CANESM2':
        mem = f'{member}'
        fl = f'{dir_in}/CO2_flux_CanESM2{mem}_SOMFFN.nc'
        
    
    ###======================================
    ### STL Decomposition
    ###======================================
    ### Load raw data
    ds = xr.open_dataset(fl)
    ds = ds.transpose('time', 'lat', 'lon')
    flux = ds[f'F_{member_or_somffn}'].copy()
       
    ### 1. Detrend it 
    flux_detrend = stl.detrend(flux, dim='time')
    
    ### 2. Remove seasonal cycle
    # 2.1 -- seasonal cycle
    flux_seasonal = stl.seasonal_cycle(flux_detrend, dim='time', period=12)
    
    # 2.2 -- de-season the data
    flux_deseason = flux_detrend - flux_seasonal
    
    ### 3. calculate the LOWESS
    flux_lowess = stl.lowess(flux_deseason, dim='time', lo_pts=12*10, lo_delta=0.01)

    ### 4. Residual term -- not explained by trend or seasonal cycle 
    flux_residual = flux_deseason - flux_lowess
    
    ### Low frequency of residual
    flux_residual_low = stl.lowess(flux_residual, dim='time', lo_pts=12, lo_delta=0.01)

    
    ###======================================
    ### Put data into a dataset
    ###======================================
    ds_out = xr.Dataset(
        {    
        f'F_{member_or_somffn}': (['time', 'lat', 'lon'], flux ),
        f'F_{member_or_somffn}_detrend': (['time', 'lat', 'lon'], flux_detrend ),
        f'F_{member_or_somffn}_dec': (['time','lat', 'lon'], flux_lowess ),
        f'F_{member_or_somffn}_seasonal': (['time','lat', 'lon'], flux_seasonal ),
        f'F_{member_or_somffn}_residual': (['time','lat', 'lon'], flux_residual ),
        f'F_{member_or_somffn}_residual_low': (['time','lat', 'lon'], flux_residual_low ),
        },

        coords={
        'time': (['time'], ds['time']),
        'lat': (['lat'], ds['lat']),
        'lon': (['lon'], ds['lon']),
        })
        
    # Save netCDF file
    ds_out.to_netcdf(f'{dir_out}/CO2_flux_decomp_{model}{mem}_{model_or_recon}.nc') 
    
    # clean up memory
    del ds, ds_out, flux, flux_detrend, flux_lowess, flux_seasonal, flux_residual, flux_residual_low

# Calculate pCO2 statistics

In [ ]:
def calculate_pco2_statistics(model=None, member=None, dir_in=None, dir_out=None):
    """
    calculate_pco2_statistics(model=None, member=None)
    model   = [MPI, CESM, or GFDL]
    member  = "member you want to process"
    dir_in  = directory for CO2 inputs (do not put / at end)
    dir_out = directory for output (do not put / at end) 
    
    Notes
    =========
    
    
    L. Gloege 2019
    """
    ### ================================================
    ### Get file path
    ### ================================================
    if model.upper()=='CESM':
        mem = f'{member:0>3}'
    if model.upper()=='GFDL':
        mem = f'{member:0>2}'
    if model.upper()=='MPI':
        mem = f'{member:0>3}'
    if model.upper()=='CANESM2':
        mem = f'{member}'
    
    ###======================================
    ### Load model
    ###====================================== 
    
    ds_som = xr.open_dataset(f'{dir_in}/pco2_decomp_{model}{mem}_SOMFFN.nc')
    ds_mod = xr.open_dataset(f'{dir_in}/pco2_decomp_{model}{mem}_MODEL.nc')
        
    ###======================================
    ### Return dataset
    ###======================================
    ds_out = xr.Dataset(
        {
        ## Bias
        'bias': (['lat', 'lon'], sk.avg_error(ds_mod['pco2'], 
                                              ds_som['pco2']),
                 {'units':'uatm'}), 
        'bias_detrend':(['lat','lon'], sk.avg_error(ds_mod['pco2_detrend'], 
                                                    ds_som['pco2_detrend']),
                       {'units':'uatm'}),
        'bias_dec': (['lat', 'lon'], sk.avg_error(ds_mod['pco2_dec'], 
                                                    ds_som['pco2_dec']),
                       {'units':'uatm'}),
        'bias_seasonal': (['lat', 'lon'], sk.avg_error(ds_mod['pco2_seasonal'], 
                                                      ds_som['pco2_seasonal']),
                       {'units':'uatm'}),
        'bias_residual': (['lat', 'lon'], sk.avg_error(ds_mod['pco2_residual'], 
                                                      ds_som['pco2_residual']),
                       {'units':'uatm'}),
        'bias_residual_low': (['lat', 'lon'], sk.avg_error(ds_mod['pco2_residual_low'], 
                                                      ds_som['pco2_residual_low']),
                       {'units':'uatm'}),

        ## average absolute error
        'aae': (['lat', 'lon'], sk.avg_abs_error(ds_mod['pco2'], 
                                              ds_som['pco2']),
                 {'units':'uatm'}), 
        'aae_detrend':(['lat','lon'], sk.avg_abs_error(ds_mod['pco2_detrend'], 
                                                    ds_som['pco2_detrend']),
                       {'units':'uatm'}),
        'aae_dec': (['lat', 'lon'], sk.avg_abs_error(ds_mod['pco2_dec'], 
                                                    ds_som['pco2_dec']),
                       {'units':'uatm'}),
        'aae_seasonal': (['lat', 'lon'], sk.avg_abs_error(ds_mod['pco2_seasonal'], 
                                                      ds_som['pco2_seasonal']),
                       {'units':'uatm'}),
        'aae_residual': (['lat', 'lon'], sk.avg_abs_error(ds_mod['pco2_residual'], 
                                                      ds_som['pco2_residual']),
                       {'units':'uatm'}),
        'aae_residual_low': (['lat', 'lon'], sk.avg_abs_error(ds_mod['pco2_residual_low'], 
                                                      ds_som['pco2_residual_low']),
                       {'units':'uatm'}),
            
       
        ## Amplitude ratio
        'amp-ratio': (['lat', 'lon'], sk.amp_ratio(ds_mod['pco2'], 
                                              ds_som['pco2']),
              {'units':'dimensionless',
               'notes':'[(amp_rec/amp_mod) - 1]'}),
        'amp-ratio_detrend':(['lat','lon'], sk.amp_ratio(ds_mod['pco2_detrend'], 
                                                    ds_som['pco2_detrend']),
              {'units':'dimensionless',
               'notes':'[(amp_rec/amp_mod) - 1]'}),
        'amp-ratio_dec': (['lat', 'lon'], sk.amp_ratio(ds_mod['pco2_dec'], 
                                                    ds_som['pco2_dec']),
              {'units':'dimensionless',
               'notes':'[(amp_rec/amp_mod) - 1]'}),
        'amp-ratio_seasonal': (['lat', 'lon'], sk.amp_ratio(ds_mod['pco2_seasonal'], 
                                                      ds_som['pco2_seasonal']),
              {'units':'dimensionless',
               'notes':'[(amp_rec/amp_mod) - 1]'}),
        'amp-ratio_residual': (['lat', 'lon'], sk.amp_ratio(ds_mod['pco2_residual'], 
                                                      ds_som['pco2_residual']),
              {'units':'dimensionless',
               'notes':'[(amp_rec/amp_mod) - 1]'}),
        'amp-ratio_residual_low': (['lat', 'lon'], sk.amp_ratio(ds_mod['pco2_residual_low'], 
                                                      ds_som['pco2_residual_low']),
              {'units':'dimensionless',
               'notes':'[(amp_rec/amp_mod) - 1]'}),
            
        
        ## correlation
        'corr': (['lat', 'lon'], sk.correlation(ds_mod['pco2'], 
                                              ds_som['pco2']),
            {'units':'dimensionless'}),
        'corr_detrend':(['lat','lon'], sk.correlation(ds_mod['pco2_detrend'], 
                                                    ds_som['pco2_detrend']),
            {'units':'dimensionless'}),
        'corr_dec': (['lat', 'lon'], sk.correlation(ds_mod['pco2_dec'], 
                                                    ds_som['pco2_dec']),
            {'units':'dimensionless'}),
        'corr_seasonal': (['lat', 'lon'], sk.correlation(ds_mod['pco2_seasonal'], 
                                                      ds_som['pco2_seasonal']),
            {'units':'dimensionless'}),
        'corr_residual': (['lat', 'lon'], sk.correlation(ds_mod['pco2_residual'], 
                                                      ds_som['pco2_residual']),
            {'units':'dimensionless'}),
        'corr_residual_low': (['lat', 'lon'], sk.correlation(ds_mod['pco2_residual_low'], 
                                                      ds_som['pco2_residual_low']),
            {'units':'dimensionless'}),
            
            
        ## Normalized standard deviation
        'std-star': (['lat', 'lon'], sk.std_star(ds_mod['pco2'], 
                                              ds_som['pco2']),
                      {'units':'dimensionless',
                       'notes':'[(STD_rec/STD_mod) - 1]'}),
        'std-star_detrend':(['lat','lon'], sk.std_star(ds_mod['pco2_detrend'], 
                                                    ds_som['pco2_detrend']),
                      {'units':'dimensionless',
                       'notes':'[(STD_rec/STD_mod) - 1]'}),
        'std-star_dec': (['lat', 'lon'], sk.std_star(ds_mod['pco2_dec'], 
                                                    ds_som['pco2_dec']),
                      {'units':'dimensionless',
                       'notes':'[(STD_rec/STD_mod) - 1]'}),
        'std-star_seasonal': (['lat', 'lon'], sk.std_star(ds_mod['pco2_seasonal'], 
                                                      ds_som['pco2_seasonal']),
                      {'units':'dimensionless',
                       'notes':'[(STD_rec/STD_mod) - 1]'}),
        'std-star_residual': (['lat', 'lon'], sk.std_star(ds_mod['pco2_residual'], 
                                                      ds_som['pco2_residual']),
                      {'units':'dimensionless',
                       'notes':'[(STD_rec/STD_mod) - 1]'}),
        'std-star_residual_low': (['lat', 'lon'], sk.std_star(ds_mod['pco2_residual_low'], 
                                                      ds_som['pco2_residual_low']),
                      {'units':'dimensionless',
                       'notes':'[(STD_rec/STD_mod) - 1]'})
        },

        coords={
        'lat': (['lat'], ds_mod['lat']),
        'lon': (['lon'], ds_mod['lon']),
        })
        
    # save file
    ds_out.to_netcdf(f'{dir_out}/stats_pco2_{model}{mem}_SOMFFN.nc') 

# Old stuff

In [ ]:
    ###======================================
    ### Decompose the time series
    ###======================================
    #pco2_model = ds['pco2_model'].copy()
    #pco2_model_detrend = pr(pco2_model).detrend().values
    #pco2_model_iav = pr(pco2_model_detrend).rolling_mean(window=12).values
    #pco2_model_av = pco2_model_detrend - pco2_model_iav
    #pco2_model_dec = pr(pco2_model_iav).rolling_mean(window=60).values
    #pco2_model_subdec = pco2_model_iav - pco2_model_dec

In [ ]:
def decompose_flux_OLD(model=None, member=None, model_or_recon=None, dir_in=None, dir_out=None):
    '''
    decompose_flux
    this function decomposes flux into various time scales
    
    Inputs
    ============
    model  = name of model (CESM, CanESM2, GFDL, MPI)
    member = member number
    model_or_recon = either SOMFFN or MODEL
    dir_in  = directory for CO2 inputs (do not put / at end)
    dir_out = directory for output (do not put / at end) 
    
    Output
    ============
    None. This stores the calculated flux as a NetCDF file in the {data_dir} location.
    
    Notes
    ============
    This assumes ERA interm is here:
    /local/data/artemis/workspace/gloege/SOCAT-LE/data/clean/ERA_interim/
    You will need to change this if this is not correct
    
    and data is output to this directory:
    /local/data/artemis/workspace/gloege/SOCAT-LE/data/clean/CO2_flux/
    
    '''
    
    ###======================================
    ### This is so naming convention same as decompose_pco2
    ###====================================== 
    if model_or_recon.upper() == 'MODEL':
        member_or_somffn = 'member'
    elif model_or_recon.upper() == 'SOMFFN':
        member_or_somffn = 'somffn'
    else:
        print('model_or_recon needs to be either MODEL or SOMFFN')
        
    ###======================================
    ### Define directories
    ###====================================== 
    #dir_clean = '/local/data/artemis/workspace/gloege/SOCAT-LE/data/clean'
    # {dir_clean}/CO2_flux
    
    ###======================================
    ### Load data
    ###======================================
    if model.upper()=='CESM':
        mem = f'{member:0>3}'
        fl = f'{dir_in}/CO2_flux_CESM{mem}_SOMFFN.nc'
        
    if model.upper()=='GFDL':
        mem = f'{member:0>2}'
        fl = f'{dir_in}/CO2_flux_GFDL{mem}_SOMFFN.nc'
        
    if model.upper()=='MPI':
        mem = f'{member:0>3}'
        fl = f'{dir_in}/CO2_flux_MPI{mem}_SOMFFN.nc'

    if model.upper()=='CANESM2':
        mem = f'{member}'
        fl = f'{dir_in}/CO2_flux_CanESM2{mem}_SOMFFN.nc'
        
    ds = xr.open_dataset(fl)
        
    ###======================================
    ### Breakdown the signal and 
    ### Decompose pCO2 into T and nonT
    ###======================================
    flux = ds[f'F_{member_or_somffn}'].copy()
    flux_detrend = pr(flux).detrend().values
    flux_iav = pr(flux_detrend).rolling_mean(window=12).values
    flux_av = flux_detrend - flux_iav
    flux_dec = pr(flux_iav).rolling_mean(window=60).values
    flux_subdec = flux_iav - flux_dec

    ###======================================
    ### Put data into a dataset
    ###======================================
    ds_out = xr.Dataset(
        {    
        f'F_{member_or_somffn}': (['time', 'lat', 'lon'], flux ),
        f'F_{member_or_somffn}_detrend': (['time', 'lat', 'lon'], flux_detrend ),
        f'F_{member_or_somffn}_iav': (['time','lat', 'lon'], flux_iav ),
        f'F_{member_or_somffn}_dec': (['time','lat', 'lon'], flux_dec ),
        f'F_{member_or_somffn}_subdec': (['time','lat', 'lon'], flux_subdec ),
        f'F_{member_or_somffn}_av': (['time','lat', 'lon'], flux_av ),
        },

        coords={
        'time': (['time'], ds['time']),
        'lat': (['lat'], ds['lat']),
        'lon': (['lon'], ds['lon']),
        })
        
    # save file {dir_clean}/CO2_flux_decomp
    ds_out.to_netcdf(f'{dir_out}/CO2_flux_decomp_{model}{mem}_{model_or_recon}.nc') 
    
    # remove what you don't need
    del ds, ds_out, flux, flux_detrend, flux_iav, flux_av, flux_dec, flux_subdec

In [ ]:
def calculate_flux_statistics_OLD(model=None, member=None, dir_in=None, dir_out=None):
    """
    calculate_flux_statistics(model=None, member=None)
    model   = [MPI, CESM, or GFDL]
    member  = "member you want to process"
    dir_in  = directory for CO2 inputs (do not put / at end)
    dir_out = directory for output (do not put / at end) 
    
    Notes
    =========
    
    
    L. Gloege 2018
    """
    ###======================================
    ### Define directories
    ###====================================== 
    #dir_clean = '/local/data/artemis/workspace/gloege/SOCAT-LE/data/clean'
    
    ### ================================================
    ### Get file path
    ### ================================================
    if model.upper()=='CESM':
        mem = f'{member:0>3}'
    if model.upper()=='GFDL':
        mem = f'{member:0>2}'
    if model.upper()=='MPI':
        mem = f'{member:0>3}'
    if model.upper()=='CANESM2':
        mem = f'{member}'
        
    ###======================================
    ### Load model
    ###====================================== 
    ds = xr.open_dataset(f'{dir_in}/CO2_flux_{model}{mem}_SOMFFN.nc')
    
    ###======================================
    ### Breakdown the signal and 
    ### _iav = detrended with 12 month running mean
    ### _dec = 60 month running mean applied to _iav
    ### _subdec = _dec removed from _iav
    ### _av = _iav removed from detrended pco2
    ###======================================
    ### SOMFFN breakdown signal
    flux_recon = ds['F_somffn'].copy()
    flux_recon_iav = pr(flux_recon).detrend().rolling_mean(window=12).values
    flux_recon_dec = pr(flux_recon_iav).rolling_mean(window=60).values
    flux_recon_subdec = flux_recon_iav - flux_recon_dec
    flux_recon_av = pr(flux_recon).detrend().values - flux_recon_iav

    ### model breakdown
    flux_model = ds['F_member'].copy()
    flux_model_iav = pr(flux_model).detrend().rolling_mean(window=12).values
    flux_model_dec = pr(flux_model_iav).rolling_mean(window=60).values
    flux_model_subdec = flux_model_iav - flux_model_dec
    flux_model_av = pr(flux_model).detrend().values - flux_model_iav

    ###======================================
    ### Return dataset
    ###======================================
    ds_out = xr.Dataset(
        {
        # Bias
        'bias': (['lat', 'lon'], sk.avg_error(flux_model, flux_recon),
                 {'units':'mol/m2/yr'}), 
        'bias_detrend':(['lat','lon'], sk.avg_error(pr(flux_model).detrend().values, 
                                                    pr(flux_recon).detrend().values),
                       {'units':'mol/m2/yr'}),
        'bias_iav': (['lat', 'lon'], sk.avg_error(flux_model_iav, flux_recon_iav),
                    {'units':'mol/m2/yr'}),
        'bias_dec': (['lat', 'lon'], sk.avg_error(flux_model_dec, flux_recon_dec),
                    {'units':'mol/m2/yr'}),
        'bias_subdec': (['lat', 'lon'], sk.avg_error(flux_model_subdec, flux_recon_subdec),
                       {'units':'mol/m2/yr'}),
        'bias_av': (['lat', 'lon'], sk.avg_error(flux_model_av, flux_recon_av),
                   {'units':'mol/m2/yr'}),
      
        # Average absolute error
        'aae':(['lat','lon'], sk.avg_abs_error(flux_model,flux_recon),
              {'units':'mol/m2/yr'}),
        'aae_detrend':(['lat','lon'], sk.avg_abs_error(pr(flux_model).detrend().values, 
                                                       pr(flux_recon).detrend().values),
                      {'units':'uatm'}),
        'aae_iav':(['lat','lon'], sk.avg_abs_error(flux_model_iav, flux_recon_iav),
                  {'units':'mol/m2/yr'}),
        'aae_dec':(['lat','lon'], sk.avg_abs_error(flux_model_dec, flux_recon_dec),
                  {'units':'mol/m2/yr'}),
        'aae_subdec':(['lat','lon'], sk.avg_abs_error(flux_model_subdec, flux_recon_subdec),
                     {'units':'mol/m2/yr'}),
        'aae_av':(['lat','lon'], sk.avg_abs_error(flux_model_av, flux_recon_av),
                 {'units':'mol/m2/yr'}),
       
        # amp_ratio
        'amp-ratio':(['lat','lon'], sk.amp_ratio(flux_model,flux_recon),
              {'units':'dimensionless',
               'notes':'[(amp_rec/amp_mod) - 1]'}),
        'amp-ratio_detrend':(['lat','lon'], sk.amp_ratio(pr(flux_model).detrend().values, 
                                                       pr(flux_recon).detrend().values),
                      {'units':'dimensionless',
                       'notes':'[(amp_rec/amp_mod) - 1]'}),
        'amp-ratio_iav':(['lat','lon'], sk.amp_ratio(flux_model_iav, flux_recon_iav),
                  {'units':'dimensionless',
                  'notes':'[(amp_rec/amp_mod) - 1]'}),
        'amp-ratio_dec':(['lat','lon'], sk.amp_ratio(flux_model_dec, flux_recon_dec),
                  {'units':'dimensionless',
                  'notes':'[(amp_rec/amp_mod) - 1]'}),
        'amp-ratio_subdec':(['lat','lon'], sk.amp_ratio(flux_model_subdec, flux_recon_subdec),
                     {'units':'dimensionless',
                     'notes':'[(amp_rec/amp_mod) - 1]'}),
        'amp-ratio_av':(['lat','lon'], sk.amp_ratio(flux_model_av, flux_recon_av),
                 {'units':'dimensionless',
                 'notes':'[(amp_rec/amp_mod) - 1]'}),
            
        # Correlation
        'corr':(['lat','lon'], sk.correlation(flux_model, flux_recon),
               {'units':'dimensionless'}),
        'corr_detrend':(['lat','lon'], sk.correlation(pr(flux_model).detrend().values, 
                                                      pr(flux_recon).detrend().values),
                       {'units':'dimensionless'}),
        'corr_iav':(['lat','lon'], sk.correlation(flux_model_iav, flux_recon_iav),
                   {'units':'dimensionless'}),
        'corr_dec':(['lat','lon'], sk.correlation(flux_model_dec, flux_recon_dec),
                   {'units':'dimensionless'}),
        'corr_subdec':(['lat','lon'], sk.correlation(flux_model_subdec, flux_recon_subdec),
                      {'units':'dimensionless'}),
        'corr_av':(['lat','lon'], sk.correlation(flux_model_av, flux_recon_av),
                  {'units':'dimensionless'}),
         
        # Normalized standard deviation
        'std-star':(['lat','lon'], sk.std_star(flux_model, flux_recon),
                   {'units':'dimensionless', 
                    'notes':'[(STD_rec/STD_mod) - 1]'}),
        'std-star_detrend':(['lat','lon'], sk.std_star(pr(flux_model).detrend().values, 
                                                       pr(flux_recon).detrend().values),
                           {'units':'dimensionless', 
                            'notes':'[(STD_rec/STD_mod) - 1]'}),
        'std-star_iav':(['lat','lon'], sk.std_star(flux_model_iav, flux_recon_iav),
                       {'units':'dimensionless',
                        'notes':'[(STD_rec/STD_mod) - 1]'}),
        'std-star_dec':(['lat','lon'], sk.std_star(flux_model_dec, flux_recon_dec),
                       {'units':'dimensionless',
                        'notes':'[(STD_rec/STD_mod) - 1]'}),
        'std-star_subdec':(['lat','lon'], sk.std_star(flux_model_subdec, flux_recon_subdec),
                          {'units':'dimensionless',
                           'notes':'[(STD_rec/STD_mod) - 1]'}),
        'std-star_av':(['lat','lon'], sk.std_star(flux_model_av, flux_recon_av),
                      {'units':'dimensionless',
                       'notes':'[(STD_rec/STD_mod) - 1]'})
        },

        coords={
        'lat': (['lat'], ds['lat']),
        'lon': (['lon'], ds['lon']),
        })
        
    # save file
    ds_out.to_netcdf(f'{dir_out}/stats_FCO2_{model}{mem}_SOMFFN.nc')